In [2]:
%set_env DBIKE_DEV=True
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
import web.db_utils
import datetime

env: DBIKE_DEV=True
True
2024-04-04 14:20:33,045 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-04 14:20:33,046 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 14:20:33,419 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-04 14:20:33,420 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 14:20:33,606 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-04 14:20:33,607 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 14:20:33,991 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 14:20:33,992 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `Station`
2024-04-04 14:20:33,993 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 14:20:34,088 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-04 14:20:34,369 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 14:20:34,370 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CurrentWeather`
2024-04-04 14:20:34,371 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 14:20:34,468 INFO sqlal

In [3]:
DAYS_BACK = 30
FEATURE_COLUMNS = [
        "FeelsLike",
        "Humidity",
        "Pressure",
        "Temperature",
        "day",
        "month",
        "hour",
        "is_weekday",
        "minute",
        "cold_weather",
        "windy_weather"
    ]
TARGET_COLUMNS = ["StandsAvailable"]

In [4]:
# Load the datasets using Pandas
weather = web.db_utils.get_historical_weather(30)
weather_data = pd.DataFrame(weather)


2024-04-04 14:20:35,819 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 14:20:35,819 INFO sqlalchemy.engine.Engine SELECT `CurrentWeather`.`DateTime`, `CurrentWeather`.`FeelsLike`, `CurrentWeather`.`Humidity`, `CurrentWeather`.`Pressure`, `CurrentWeather`.`Sunrise`, `CurrentWeather`.`Sunset`, `CurrentWeather`.`Temperature`, `CurrentWeather`.`UVI`, `CurrentWeather`.`WeatherId`, `CurrentWeather`.`WindGust`, `CurrentWeather`.`WindSpeed`, `CurrentWeather`.`Rain1h`, `CurrentWeather`.`Snow1h` 
FROM `CurrentWeather` 
WHERE DATE(`CurrentWeather`.`DateTime`) >= %(DATE_1)s
2024-04-04 14:20:35,820 INFO sqlalchemy.engine.Engine [generated in 0.09397s] {'DATE_1': datetime.datetime(2024, 3, 5, 14, 20, 35, 723222)}
2024-04-04 14:20:36,108 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
# Preprocess the data
weather_data['time'] = pd.to_datetime(weather_data['DateTime'])

weather_data['month'] = weather_data['time'].dt.month
weather_data['day'] = weather_data['time'].dt.day
weather_data['hour'] = weather_data['time'].dt.hour
weather_data['minute'] = weather_data['time'].dt.minute
weather_data['is_weekday'] = ((weather_data['time'].dt.weekday >= 0) & (weather_data['time'].dt.weekday <= 4)).astype(int)

weather_data.head(10)


,DateTime,FeelsLike,Humidity,Pressure,Sunrise,Sunset,Temperature,UVI,WeatherId,WindGust,WindSpeed,Rain1h,Snow1h,time,month,day,hour,minute,is_weekday
0,2024-03-07 07:40:46,3.64,85,1019,2024-03-07 06:58:06,2024-03-07 18:14:13,7.06,None,803,None,5.66,None,None,2024-03-07 07:40:46,3,7,7,40,1
1,2024-03-07 08:04:06,3.80,85,1019,2024-03-07 06:58:06,2024-03-07 18:14:14,7.19,None,803,None,5.66,None,None,2024-03-07 08:04:06,3,7,8,4,1
2,2024-03-07 08:09:47,4.20,85,1019,2024-03-07 06:58:05,2024-03-07 18:14:13,7.34,None,803,None,5.14,None,None,2024-03-07 08:09:47,3,7,8,9,1
3,2024-03-07 09:00:04,4.29,84,1020,2024-03-07 06:58:05,2024-03-07 18:14:13,7.48,None,803,None,5.36,None,None,2024-03-07 09:00:04,3,7,9,0,1
4,2024-03-07 10:00:11,4.36,82,1020,2024-03-07 06:58:05,2024-03-07 18:14:13,7.92,None,803,None,6.71,None,None,2024-03-07 10:00:11,3,7,10,0,1
5,2024-03-07 11:00:08,4.62,80,1020,2024-03-07 06:58:05,2024-03-07 18:14:13,8.36,None,803,None,7.72,None,None,2024-03-07 11:00:08,3,7,11,0,1
6,2024-03-07 12:00:05,5.15,78,1019,2024-03-07 06:58:05,2024-03-07 18:14:13,8.53,None,803,None,6.69,None,None,2024-03-07 12:00:05,3,7,12,0,1
7,2024-03-07 13:00:07,4.98,76,1019,2024-03-07 06:58:05,2024-03-07 18:14:13,8.52,None,803,None,7.20,None,None,2024-03-07 13:00:07,3,7,13,0,1
8,2024-03-07 14:00:07,4.15,77,1018,2024-03-07 06:58:05,2024-03-07 18:14:13,8.00,None,802,None,7.72,None,None,2024-03-07 14:00:07,3,7,14,0,1
9,2024-03-07 15:00:08,4.71,73,1018,2024-03-07 06:58:05,2024-03-07 18:14:13,8.43,None,802,None,7.72,None,None,2024-03-07 15:00:08,3,7,15,0,1


In [6]:
# Extract day and hour
stations = web.db_utils.get_stations()
station_ids = [s["Id"] for s in stations]


2024-04-04 14:20:36,346 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 14:20:36,347 INFO sqlalchemy.engine.Engine SELECT `Station`.`Id`, `Station`.`Name`, `Station`.`PositionLatitude`, `Station`.`PositionLongitude`, `Station`.`Address`, `Station`.`City`, `Station`.`AcceptsCard`, `Station`.`TotalStands` 
FROM `Station`
2024-04-04 14:20:36,347 INFO sqlalchemy.engine.Engine [generated in 0.09420s] {}
2024-04-04 14:20:36,446 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
def to_date(date_str):
    try:
        dt = datetime.datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')
        return dt
    except:
        dt = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
        return dt

In [9]:
import logging

# Disable SQLAlchemy INFO logs
logging.getLogger('sqlalchemy.engine').setLevel(logging.ERROR)
mses = []
for id in station_ids:
    availability = web.db_utils.get_historical_availabilities(id, DAYS_BACK)
    station_data = pd.DataFrame(availability)
    station_data['last_updated_dt'] = station_data['LastUpdated'].apply(to_date)
    station_data['month'] = station_data['last_updated_dt'].dt.month
    station_data['day'] = station_data['last_updated_dt'].dt.day
    station_data['hour'] = station_data['last_updated_dt'].dt.hour
    station_data['minute'] = station_data['last_updated_dt'].dt.minute

    merged_data = pd.merge(station_data, weather_data, on = ['month','day','hour', 'minute'])
    merged_data['is_busy_hours'] = ((merged_data['hour'] >= 7) & (merged_data['hour'] <= 10)) | ((merged_data['hour'] >= 16) & (merged_data['hour'] <= 19)).astype(int)
    merged_data['cold_weather'] = (merged_data['Temperature'] < 5).astype(float)
    merged_data['windy_weather'] = (merged_data['WindSpeed'] > 8).astype(float)


    # Split the data into training and testing sets
    X = merged_data[FEATURE_COLUMNS]
    y = merged_data[TARGET_COLUMNS]

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Train a machine learning model
    # model = LinearRegression()
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    # Serialize the trained model into a file called model.pkl
    with open(f'models/linear-v1/model_{id}.pkl', 'wb') as handle:
        pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)

    # Deserialize the model.pkl file into an object called model
    # with open(f'model_{id}.pkl', 'rb') as handle:
    #     model = pickle.load(handle)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mses.append(f"Mean Squared Error for station {id}: {mse}")

for m in mses:
    print(m)
    

2024-04-04 14:37:58,816 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 14:37:58,817 INFO sqlalchemy.engine.Engine SELECT `Availability`.`StationId`, `Availability`.`Status`, `Availability`.`MechanicalBikesAvailable`, `Availability`.`ElectricBikesAvailable`, `Availability`.`StandsAvailable`, `Availability`.`LastUpdated` 
FROM `Availability` 
WHERE `Availability`.`StationId` = %(StationId_1)s AND DATE(`Availability`.`LastUpdated`) >= %(DATE_1)s
2024-04-04 14:37:58,818 INFO sqlalchemy.engine.Engine [cached since 271.1s ago] {'StationId_1': 1, 'DATE_1': datetime.datetime(2024, 3, 5, 14, 37, 58, 692253)}
2024-04-04 14:37:59,210 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-04 14:37:59,506 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 14:37:59,506 INFO sqlalchemy.engine.Engine SELECT `Availability`.`StationId`, `Availability`.`Status`, `Availability`.`MechanicalBikesAvailable`, `Availability`.`ElectricBikesAvailable`, `Availability`.`StandsAvailable`, `Availability`.`

/Users/aprilpolubiec/anaconda3/envs/dublin-bikes/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


2024-04-04 14:37:59,630 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-04 14:37:59,913 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 14:37:59,913 INFO sqlalchemy.engine.Engine SELECT `Availability`.`StationId`, `Availability`.`Status`, `Availability`.`MechanicalBikesAvailable`, `Availability`.`ElectricBikesAvailable`, `Availability`.`StandsAvailable`, `Availability`.`LastUpdated` 
FROM `Availability` 
WHERE `Availability`.`StationId` = %(StationId_1)s AND DATE(`Availability`.`LastUpdated`) >= %(DATE_1)s
2024-04-04 14:37:59,913 INFO sqlalchemy.engine.Engine [cached since 272.1s ago] {'StationId_1': 3, 'DATE_1': datetime.datetime(2024, 3, 5, 14, 37, 59, 816144)}


/Users/aprilpolubiec/anaconda3/envs/dublin-bikes/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


2024-04-04 14:38:00,139 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-04 14:38:00,437 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 14:38:00,438 INFO sqlalchemy.engine.Engine SELECT `Availability`.`StationId`, `Availability`.`Status`, `Availability`.`MechanicalBikesAvailable`, `Availability`.`ElectricBikesAvailable`, `Availability`.`StandsAvailable`, `Availability`.`LastUpdated` 
FROM `Availability` 
WHERE `Availability`.`StationId` = %(StationId_1)s AND DATE(`Availability`.`LastUpdated`) >= %(DATE_1)s
2024-04-04 14:38:00,438 INFO sqlalchemy.engine.Engine [cached since 272.7s ago] {'StationId_1': 4, 'DATE_1': datetime.datetime(2024, 3, 5, 14, 38, 0, 328160)}


...

/Users/aprilpolubiec/anaconda3/envs/dublin-bikes/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


2024-04-04 14:38:43,929 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-04 14:38:44,191 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 14:38:44,192 INFO sqlalchemy.engine.Engine SELECT `Availability`.`StationId`, `Availability`.`Status`, `Availability`.`MechanicalBikesAvailable`, `Availability`.`ElectricBikesAvailable`, `Availability`.`StandsAvailable`, `Availability`.`LastUpdated` 
FROM `Availability` 
WHERE `Availability`.`StationId` = %(StationId_1)s AND DATE(`Availability`.`LastUpdated`) >= %(DATE_1)s
2024-04-04 14:38:44,192 INFO sqlalchemy.engine.Engine [cached since 316.4s ago] {'StationId_1': 117, 'DATE_1': datetime.datetime(2024, 3, 5, 14, 38, 44, 74134)}


/Users/aprilpolubiec/anaconda3/envs/dublin-bikes/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


2024-04-04 14:38:44,297 INFO sqlalchemy.engine.Engine ROLLBACK
Mean Squared Error for station 1: 36.76467777777777
Mean Squared Error for station 2: 14.787775000000002
Mean Squared Error for station 3: 14.277743333333335
Mean Squared Error for station 4: 4.4559299999999995
Mean Squared Error for station 5: 117.48741363636364
Mean Squared Error for station 6: 15.629980769230771
Mean Squared Error for station 7: 23.0887875
Mean Squared Error for station 8: 32.82588571428572
Mean Squared Error for station 9: 26.933592857142855
Mean Squared Error for station 10: 13.405636666666668
Mean Squared Error for station 11: 17.504160869565215
Mean Squared Error for station 12: 10.679039130434784
Mean Squared Error for station 13: 3.8095653846153854
Mean Squared Error for station 14: 40.44391333333332
Mean Squared Error for station 15: 2.8605791666666662
Mean Squared Error for station 16: 29.51241739130435
Mean Squared Error for station 17: 3.339171999999999
Mean Squared Error for station 18: 23.237

/Users/aprilpolubiec/anaconda3/envs/dublin-bikes/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
